In [1]:
#widen display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time

In [3]:
nba_stats_base_url = 'https://www.nba.com/stats/teams/traditional?PerMode=Totals'

In [8]:
seasons = ['&Season=2022-23', '&Season=2021-22', '&Season=2020-21', '&Season=2019-20'
          , '&Season=2018-19', '&Season=2017-18', '&Season=2016-17', '&Season=2015-16', '&Season=2014-15', '&Season=2013-14'
          , '&Season=2012-13', '&Season=2011-12', '&Season=2010-11', '&Season=2009-10', '&Season=2008-09', '&Season=2007-08'
          , '&Season=2006-07', '&Season=2005-06', '&Season=2004-05', '&Season=2003-04', '&Season=2002-03'
          , '&Season=2001-02', '&Season=2000-01']

In [5]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
#Vist first season to create dataframe - don't feel like manually creating the dataframe columns :)
browser.visit(nba_stats_base_url)
time.sleep(3)

html = browser.html
stats_html = BeautifulSoup(html, 'html.parser')

tables = pd.read_html(str(stats_html))

all_seasons_stats_df = tables[2]

all_seasons_stats_df.insert(54, "SEASON", "2023-24", True) 

In [7]:
all_seasons_stats_df.head()

,Unnamed: 0,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,...,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,BLKA RANK,PF RANK,PFD RANK,+/- RANK,SEASON
0,1,Boston Celtics,82,64,18,0.780,3966,9887,3601,7396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
1,2,Denver Nuggets,82,57,25,0.695,3941,9418,3610,7279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
2,2,Oklahoma City Thunder,82,57,25,0.695,3961,9847,3653,7324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
3,4,Minnesota Timberwolves,82,56,26,0.683,3961,9264,3383,6974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24
4,5,LA Clippers,82,51,31,0.622,3941,9481,3473,7108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-24


In [9]:
#loop through rest of seasons and append dataframes
for season in seasons:
    nba_season_stats_url = nba_stats_base_url + season

    browser.visit(nba_season_stats_url)
    time.sleep(3)
    
    html = browser.html
    stats_html = BeautifulSoup(html, 'html.parser')
    
    tables = pd.read_html(str(stats_html))
    
    this_season_stats_df = tables[2]
    
    #add the season for this set of stats
    this_season_stats_df.insert(54, "SEASON", season.strip('&Season='), True) 
    
    all_seasons_stats_df = all_seasons_stats_df.append(this_season_stats_df, ignore_index=True)

In [10]:
#drop unwanted columns
all_seasons_stats_df.drop(all_seasons_stats_df.iloc[:, 28:54], inplace = True, axis = 1)
all_seasons_stats_df.drop(all_seasons_stats_df.columns[0], inplace = True, axis=1)

In [11]:
all_seasons_stats_df.head()

,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,SEASON
0,Boston Celtics,82,64,18,0.780,3966,9887,3601,7396,48.7,...,3799,2207,979,557,538,304,1326,1416,930,2023-24
1,Denver Nuggets,82,57,25,0.695,3941,9418,3610,7279,49.6,...,3643,2415,1036,585,456,394,1489,1467,431,2023-24
2,Oklahoma City Thunder,82,57,25,0.695,3961,9847,3653,7324,49.9,...,3447,2223,1039,694,538,419,1545,1548,608,2023-24
3,Minnesota Timberwolves,82,56,26,0.683,3961,9264,3383,6974,48.5,...,3577,2184,1162,647,497,371,1544,1630,529,2023-24
4,LA Clippers,82,51,31,0.622,3941,9481,3473,7108,48.9,...,3523,2097,1078,640,413,384,1519,1537,269,2023-24


In [12]:
all_seasons_stats_df = all_seasons_stats_df.rename(columns={'TEAM':'team','GP':'games_played','W':'wins','L':'losses','WIN%':'win_percentage','MIN':'minutes','PTS':'points','FGM':'field_goals_made','FGA':'field_goals_attempted','FG%':'field_goal_percentage','3PM':'three_pointers_made','3PA':'three_pointers_attempted','3P%':'three_point_percentage','FTM':'free_throws_made','FTA':'free_throw_attempted','FT%':'free_throw_percentage','OREB':'offensive_rebounds','DREB':'defensive_rebounds','REB':'rebounds','AST':'assists','TOV':'turnovers','STL':'steals','BLK':'blocks','BLKA':'blocks_attempted','PF':'personal_fouls','PFD':'personal_fouls_drawn','+/-':'plus_minus','SEASON':'season'})

In [13]:
all_seasons_stats_df.index.name = 'teamstatspk'

In [14]:
all_seasons_stats_df.to_csv('data/nba_team_stats_00_to_23.csv')

In [15]:
all_seasons_stats_df.head()

,Team,games_played,wins,losses,win_percentage,Min,points,field_goals_made,field_goals_attempted,field_goal_percentage,...,rebounds,assists,turnovers,steals,blocks,blocks_attempted,personal_fouls,personal_fouls_drawn,plus_minus,season
teamstatspk,,,,,,,,,,,,,,,,,,,,,
0,Boston Celtics,82,64,18,0.780,3966,9887,3601,7396,48.7,...,3799,2207,979,557,538,304,1326,1416,930,2023-24
1,Denver Nuggets,82,57,25,0.695,3941,9418,3610,7279,49.6,...,3643,2415,1036,585,456,394,1489,1467,431,2023-24
2,Oklahoma City Thunder,82,57,25,0.695,3961,9847,3653,7324,49.9,...,3447,2223,1039,694,538,419,1545,1548,608,2023-24
3,Minnesota Timberwolves,82,56,26,0.683,3961,9264,3383,6974,48.5,...,3577,2184,1162,647,497,371,1544,1630,529,2023-24
4,LA Clippers,82,51,31,0.622,3941,9481,3473,7108,48.9,...,3523,2097,1078,640,413,384,1519,1537,269,2023-24
